# Pandas

In [3]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

# Indexación Jerárquica

Al colocar en la opción index un array multidimensional se genera una jerarquia de indices donde el arreglo correspondiente a la primera entrada es la primera capa de la jerarquia. 

In [4]:
data = Series(np.random.rand(10), 
                 index =[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'],
                 [1, 2,3, 1, 2,3, 1, 2,2, 3]])
data

a  1    0.997133
   2    0.329860
   3    0.537674
b  1    0.910829
   2    0.446875
   3    0.379329
c  1    0.968902
   2    0.771363
d  2    0.265769
   3    0.518155
dtype: float64

Nota que en el arreglo anterior, las etiquetas de la primera capa (a, b,c,d) etiquetan bloques enteros. Cada bloque tiene sus propios indices que corresponden a las entradas de la segunda capa.


Al colocar un array multidimensional como indice pandas crea un arreglo de tuplas, la entrada i-esima de la j-esima tupla corresponde al i-esimo indice de la j-esima capa. Para acceder al indice jerarquico sigue funcionando el metodo index

In [5]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

 Se puede acceder a las capas del multiindice usando 'levels' 

In [96]:
capa1,capa2=data.index.levels
capa1

Index(['a', 'b', 'c', 'd'], dtype='object')

Para acceder a los bloques etiquetados por la primera capa de indices se usa la misma notación que para series o arreglos de numpy

In [6]:
data['b':'c']

b  1    0.910829
   2    0.446875
   3    0.379329
c  1    0.968902
   2    0.771363
dtype: float64

Para acceder a bloques predeterminados se usa la misma notación que para acceder a las columnas de un dataframe

In [7]:
data.loc[['b', 'd']]

b  1    0.910829
   2    0.446875
   3    0.379329
d  2    0.265769
   3    0.518155
dtype: float64

Nota que una Serie con indices jerarquicos de orden 2 es equivalente a un dataframe.


In [8]:
data[:, 2]

a    0.329860
b    0.446875
c    0.771363
d    0.265769
dtype: float64

## Una serie con índices jerárquicos puede convertirse en dataframe usando el metodo unstack 

In [9]:
# arrange it into a DataFrame

data.unstack()

,1,2,3
a,0.997133,0.329860,0.537674
b,0.910829,0.446875,0.379329
c,0.968902,0.771363,NaN
d,NaN,0.265769,0.518155


## Y deshacer tal acción con stack

In [12]:
data.unstack().stack()

a  1    0.237395
   2    0.129876
   3    0.435973
b  1    0.403339
   2    0.628142
   3    0.261277
c  1    0.218988
   2    0.229456
d  2    0.163585
   3    0.280751
dtype: float64

### Tanto las filas como las columnas pueden tener indices jerárquicos

In [17]:
frame = DataFrame(np.arange(12).reshape((4,3)),
                     index = [['a', 'a', 'b', 'b'], [1, 2, 'I', 'II']],
                     columns = [['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])


In [18]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    I        6   7        8
     II       9  10       11

In [19]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    I         6    7
     II        9   10

# Dada una serie pueden asociarsele un indice jerarquico con MultiIndex

## A partir de arrays

In [21]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                             names =['state', 'color'])


MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

## y a partir de tuplas 

In [24]:
multiindex = pd.MultiIndex.from_tuples([("A", 1), ("A", 2), ("A", 3), ("B", 2), ("B", 3)], names=["letters", "numbers"])
multiindex

MultiIndex([('A', 1),
            ('A', 2),
            ('A', 3),
            ('B', 2),
            ('B', 3)],
           names=['letters', 'numbers'])

## y asociar ese multindice a una serie

In [27]:
muestra = pd.Series(list(range(100, 105)),index=multiindex)
muestra

letters  numbers
A        1          100
         2          101
         3          102
B        2          103
         3          104
dtype: int64

In [28]:
muestra[("B", 2)]

103

# Groupby es útil  para agrupar datos de acuerdo a algún criterio


### Por ejemplo, al agrupar los elementos minimos con en los bloques etiquetados con "letters" o numbers   se almacenan en una nueva serie 

In [33]:
muestra.groupby(level="letters").min()

letters
A    100
B    103
dtype: int64

In [32]:
muestra.groupby(level="numbers").min()

numbers
1    100
2    101
3    102
dtype: int64

### Al aplicar "groupby" se realizan combinadamente las siguientes acciones:  expandir un objeto y aplicar una función.
### Puede usarse para agrupar conjuntos de datos grandes y operarlos

In [63]:
arrays = [['Falcon', 'Falcon', 'Parrot', 'Parrot'],
          ['Captive', 'Wild', 'Captive', 'Wild']]
index = pd.MultiIndex.from_arrays(arrays, names=('Animal', 'Type'))
df = pd.DataFrame({'Max Speed': [390., 350., 30., 20.]},index=index)
df

Max Speed
Animal Type              
Falcon Captive      390.0
       Wild         350.0
Parrot Captive       30.0
       Wild          20.0

In [41]:
df.groupby(level="Type").mean()

,Max Speed
Type,
Captive,210.0
Wild,185.0


In [39]:
df.groupby(level="Animal").mean()

,Max Speed
Animal,
Falcon,370.0
Parrot,25.0


## Groupby también puede aplicarse a Dataframes

In [87]:
data = Series(np.random.rand(10), 
                 index =[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'],
                 [1, 2,3, 1, 2,3, 1, 2,2, 3]])
data

a  1    0.418911
   2    0.320825
   3    0.848043
b  1    0.734150
   2    0.149301
   3    0.582284
c  1    0.794559
   2    0.832885
d  2    0.451932
   3    0.287730
dtype: float64

In [90]:
data.groupby(1).max()

KeyError: 1

### Reordering and Sorting Levels

### swaplevel

In [27]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

### sortlevel

In [35]:
frame.sortlevel(1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [29]:
frame.swaplevel(0,1).sortlevel(1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

## Summary Statistics by Level

### level
you can specify the level you want to (sum) by on a particular axis

In [36]:
frame.sum(level = 'key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [41]:
frame.sum(level = 'color', axis= 1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## Using a DataFrame's Columns

In [44]:
frame = DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one','one', 'two', 'two', 'two', 'two'],
                      'd': [0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


### set_index

In [46]:
frame2 =  frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [47]:
# but still leave the colums

frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

### reset_index

In [48]:
# and the opposite, move an index to the colums

frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# Other pandas Topics 

### iloc
usually used when you have integers as index to avoid confusion 

In [52]:
ser3 = Series(range(3), index=[-5, 1, 3])
ser3.iloc[2]

2

In [53]:
frame = DataFrame(np.arange(6).reshape(3, 2), index =[2, 0, 1])

In [58]:
frame.iloc[0]

0    0
1    1
Name: 2, dtype: int64

In [59]:
frame.iloc[:, 0]

2    0
0    2
1    4
Name: 0, dtype: int64

## Panel data
three dimensional analogue of DataFrame

In [63]:
import pandas.io.data as web

In [72]:
pdata = pd.Panel(dict((stk, web.get_data_yahoo(stk))
                     for stk in ['AAPL', 'GOOG', 'MSFT', 'DELL']))
pdata

<class 'pandas.core.panel.Panel'>
Dimensions: 4 (items) x 1611 (major_axis) x 6 (minor_axis)
Items axis: AAPL to MSFT
Major_axis axis: 2010-01-04 00:00:00 to 2016-04-28 00:00:00
Minor_axis axis: Open to Adj Close

In [73]:
pdata = pdata.swapaxes('items', 'minor')

In [74]:
pdata['Adj Close']

,AAPL,DELL,GOOG,MSFT
Date,,,,
2010-01-04,28.313195,14.06528,313.062468,26.227603
2010-01-05,28.362145,14.38450,311.683844,26.236076
2010-01-06,27.911008,14.10397,303.826685,26.075067
2010-01-07,27.859412,14.23940,296.753749,25.803894
2010-01-08,28.044630,14.36516,300.709808,25.981851
...,...,...,...,...
2016-04-22,105.680000,NaN,718.770020,51.779999
2016-04-25,105.080002,NaN,723.150024,52.110001
2016-04-26,104.349998,NaN,708.140015,51.439999


In [76]:
pdata.ix[:, '6/1/2012', :]

,Open,High,Low,Close,Volume,Adj Close
AAPL,569.159996,572.650009,560.520012,560.989983,130246900.0,74.218116
DELL,12.150000,12.300000,12.045000,12.070000,19397600.0,11.675920
GOOG,571.790972,572.650996,568.350996,570.981000,6138700.0,285.205295
MSFT,28.760000,28.959999,28.440001,28.450001,56634300.0,25.598227


In [77]:
pdata.ix['Adj Close', '5/22/2012':, :]

,AAPL,DELL,GOOG,MSFT
Date,,,,
2012-05-22,73.686282,14.58765,300.100412,26.776915
2012-05-23,75.484211,12.08221,304.426106,26.192070
2012-05-24,74.790973,12.04351,301.528978,26.156079
2012-05-25,74.390105,12.05319,295.470050,26.147081
2012-05-28,NaN,12.05319,NaN,NaN
...,...,...,...,...
2016-04-22,105.680000,NaN,718.770020,51.779999
2016-04-25,105.080002,NaN,723.150024,52.110001
2016-04-26,104.349998,NaN,708.140015,51.439999


### to_frame, to_panel

In [79]:
stacked = pdata.ix[:, '5/30/2012':, :].to_frame()
stacked

Open        High         Low       Close       Volume  \
Date       minor                                                                
2012-05-30 AAPL   569.199997  579.989990  566.559990  579.169998  132357400.0   
           DELL    12.590000   12.700000   12.460000   12.560000   19787800.0   
           GOOG   588.161028  591.901014  583.530999  588.230992    3827600.0   
           MSFT    29.350000   29.480000   29.120001   29.340000   41585500.0   
2012-05-31 AAPL   580.740021  581.499985  571.460022  577.730019  122918600.0   
...                      ...         ...         ...         ...          ...   
2016-04-27 GOOG   707.289978  708.979980  692.364990  705.840027    3086700.0   
           MSFT    51.480000   51.500000   50.549999   50.939999   43145900.0   
2016-04-28 AAPL    97.610001   97.879997   94.250000   94.830002   81990700.0   
           GOOG   708.260010  714.169983  689.549988  691.020020    2851100.0   
           MSFT    50.619999   50.770000   49.560001   49.900002   42758200.0   

                   Adj Close  
Date       minor              
2012-05-30 AAPL    76.623304  
           DELL    12.149920  
           GOOG   293.821674  
           MSFT    26.399015  
2012-05-31 AAPL    76.432797  
...                      ...  
2016-04-27 GOOG   705.840027  
           MSFT    50.939999  
2016-04-28 AAPL    94.830002  
           GOOG   691.020020  
           MSFT    49.900002  

[3325 rows x 6 columns]